## Utils

In [14]:
import requests
from collections import defaultdict
import time
from tqdm import tqdm
from urllib.parse import urlparse, urljoin, quote, unquote
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import re
import json
import pandas as pd

In [15]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Cache-Control': 'no-cache',
    'Cookie': '__ac_signature=_02B4Z6wo00f01T-HHugAAIDBye4Xzinnji0.oxpAACpTqwwjWLMk6bYzWLq1LtKuCxyf4JBnFhutjSfOO-1Qwkfs5Eb6UA8XRbArPLn.ajpzuokeM02jg7MPo5ir-aX2DB0df2hc905oNieXe5; tt_webid=7327711223528130098; s_v_web_id=verify_lrs19dzr_i4Jc90TD_didi_4mhT_BZkT_cm5gJMydzolH; _ga=GA1.1.848224349.1706115740; local_city_cache=%E4%B8%8A%E6%B5%B7; csrftoken=26facb13840df006a807db7e3ea11d29; ttwid=1%7Cazraq8pdkWUY8Ds9vKuJnhFIhIfGydiRMFf9Xz8APqc%7C1706122494%7C90cabbf85e4a3ed64c09b0ae4187c86258094d03f47e2faa3a35e84d4f29215a; msToken=0z-fD5Xct0ueqSdAN_ZNDQVuRxTH0fA7G2DCAOz_KkB7FKqzd-ctdgEfBoJQhaBxcMYGUVm2H4QmZjcVJOzlzfxjcU9gGi_c39kcD_dL; tt_scid=I0CVS8GgcoEBAmjZ9d9LSONDV1qQgLfy9nFmCycmrQ0YFSlLMU9Uv6g83X.JfBGuf46b; _ga_QEHZPBE5HH=GS1.1.1706147225.3.1.1706147291.0.0.0',
    'Pragma': 'no-cache',
    'Referer': 'https://www.toutiao.com/',
    'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120", "Microsoft Edge";v="120"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'
}

In [20]:
channel_ids = {

  '财经': 3189399007,

  '科技': 3189398999,

  '热点': 3189398996,

  '国际': 3189398968,

  '军事': 3189398960,

  '体育': 3189398957,

  '娱乐': 3189398972,

  '历史': 3189398965,

  '美食': 3189399002,

  '游戏': 3189398995,

  '旅游': 3189398983,

  '养生': 3189398959,

  '时尚': 3189398984,

  '育儿': 3189399004,

}

## Get Article List

In [17]:
dataset = defaultdict(list)

In [48]:
count = 500
max_behot_time = 1706184766  

try:
    while True:
        for channel in channel_ids:
            if len(dataset[channel]) < count:
                res_json = requests.get(f'https://www.toutiao.com/api/pc/list/feed?channel_id={channel_ids[channel]}&category=pc_profile_channel', headers=headers).json()
                dataset[channel] += res_json['data']
                print(channel, len(dataset[channel]), [item['title'] for item in res_json['data']], end='\r')

        
        if all(len(dataset[channel]) >= count for channel in channel_ids):
            break
        
        max_behot_time -= len(dataset) // 10

except KeyboardInterrupt:
    pass

In [49]:
df = pd.concat([
    pd.DataFrame(dataset[channel])[['title', 'url']].assign(category=channel)
    for channel in channel_ids.keys()
]).drop_duplicates(subset=['url']).dropna()
df = df[df['url'].str.contains('toutiao.com/group')].reset_index(drop=True)
df['url'] = df['url'].str.replace('https://toutiao.com/group/', 'https://www.toutiao.com/article/', regex=False)
df

,title,url,category
0,"预制板房或将“全面拆除”？新规之下一切都变了,2种方式安置居民",https://www.toutiao.com/article/73268338630646...,财经
1,从七人屠宰队到如今员工三万人！周连奎和他的新程金锣！,https://www.toutiao.com/article/73272141988601...,财经
2,股票起飞前，“MACD”都会出现这样的特征：一旦遇到，果断满仓干,https://www.toutiao.com/article/73279525678524...,财经
3,太原各大汽车客运站车辆已全部停运,https://www.toutiao.com/article/73257044033175...,财经
4,A股：太漂亮了，八年一轮回，这轮牛市有多大？,https://www.toutiao.com/article/73280187160362...,财经
...,...,...,...
6945,Fendi 芬迪2024春夏高级定制时装秀,https://www.toutiao.com/article/73282363380869...,时尚
6946,小姐姐锁骨彩色纹身清洗记录，沉浸式解压洗纹身,https://www.toutiao.com/article/73282771092930...,时尚
6947,我马上就做95这就95骗你，我马上弄96 其实是,https://www.toutiao.com/article/73282856991642...,时尚
6948,福气满满过龙年，天津陆家嘴年货节开始啦～#天津陆家嘴 #年货节,https://www.toutiao.com/article/73282677278592...,时尚


## Get Article Text

In [50]:
df['text'] = None

In [51]:
for i in tqdm(range(len(df))):
    if df.loc[i, 'text'] != None:
        continue
    article_url = df.loc[i, 'url']
    try:
        article_res = requests.get(article_url, headers=headers)
        soup = BeautifulSoup(article_res.text)
        article_content = soup.find('div', class_='article-content')
        df.loc[i, 'text'] = article_content.get_text(separator='\n')
    except:
        continue

100%|██████████| 6950/6950 [1:24:07<00:00,  1.38it/s]


In [40]:
df = df[~df['text'].isnull()].reset_index(drop=True)
df.to_csv('toutiao.csv')